ANG NGROK TOKEN IKAW MAG PROVIDE

In [ ]:
# 1. Install the missing base library directly from GitHub
!pip install git+https://github.com/IINemo/isanlp.git

# 2. Install the RST specific library (this worked for you before, but we run it again to be safe)
!pip install isanlp_rst

# 3. Install the Deep Learning engines
!pip install tensorflow
!pip install torch

print("INSTALLATION COMPLETE. Proceed to Step 3.")

  Cloning https://github.com/IINemo/isanlp.git to /tmp/pip-req-build-7kci4ltk
  Running command git clone --filter=blob:none --quiet https://github.com/IINemo/isanlp.git /tmp/pip-req-build-7kci4ltk
  Resolved https://github.com/IINemo/isanlp.git to commit 2a102e59f9718acc7fe259dd8d83c66d5da39794
  Preparing metadata (setup.py) ... done
INSTALLATION COMPLETE. Proceed to Step 3.


In [ ]:
from isanlp_rst.parser import Parser

# Load the parser.
# If it asks to "Restart Session" again inside the log, do it, then run this cell again.
parser = Parser(hf_model_name='tchewik/isanlp_rst_v3', hf_model_version='gumrrg', cuda_device=0)

print("SUCCESS: The Robot is ready.")

SUCCESS: The Robot is ready.


In [ ]:
text = "The battery died because the voltage dropped."
result = parser(text)
print(result['rst'])

[(id=3, start=0, end=45)]


In [ ]:
text = "The battery died because the voltage dropped."
result = parser(text)

# If this prints numbers and 'relation': 'Cause', you have a working product.
print(result['rst'])

[(id=3, start=0, end=45)]


In [ ]:
# The "Decoder" Function
def print_rst_object(node, depth=0):
    indent = "    " * depth

    # 1. Get the Logic Label (Relation)
    # The library stores the relation name in a variable called 'relation'
    rel = getattr(node, 'relation', 'Root')

    # 2. Check if it is a Leaf (Text) or a Branch (Logic)
    # If it has a 'text' attribute that isn't None, it's a leaf.
    text = getattr(node, 'text', None)

    if text:
        print(f"{indent}➤ [{rel}] TEXT: {text}")
    else:
        print(f"{indent}[{rel}]")

    # 3. Go Deeper (Recursion)
    # This library usually uses 'left' and 'right' for children
    if hasattr(node, 'left') and node.left:
        print_rst_object(node.left, depth + 1)
    if hasattr(node, 'right') and node.right:
        print_rst_object(node.right, depth + 1)

# Run the decoder on the first tree in the list
print("--- DECODING RST TREE ---")
# result['rst'] is a list, so we take the first item [0]
print_rst_object(result['rst'][0])

--- DECODING RST TREE ---
➤ [causal] TEXT: The battery died because the voltage dropped.
    ➤ [elementary] TEXT: The battery died
    ➤ [elementary] TEXT: because the voltage dropped.


In [ ]:
!pip install pyngrok flask
from pyngrok import ngrok
from flask import Flask, request, jsonify
import threading

app = Flask(__name__)

# --- THE TRANSLATOR FUNCTION ---
# This converts the complex "Object" into simple "JSON" text
def tree_to_dict(node):
    # Base case: It's just text (Leaf)
    if hasattr(node, 'text') and node.text:
        return {
            "type": "leaf",
            "relation": getattr(node, 'relation', 'elementary'),
            "text": node.text
        }

    # Recursive case: It has children (Branch)
    children = []
    if hasattr(node, 'left') and node.left:
        children.append(tree_to_dict(node.left))
    if hasattr(node, 'right') and node.right:
        children.append(tree_to_dict(node.right))

    return {
        "type": "branch",
        "relation": getattr(node, 'relation', 'span'),
        "children": children
    }

# --- THE SERVER ---
@app.route('/parse', methods=['POST'])
def parse_text():
    try:
        data = request.json
        text = data.get('text', '')

        # 1. Run the AI
        result = parser(text)

        # 2. Get the first tree from the result
        rst_tree_object = result['rst'][0]

        # 3. Translate it to JSON
        json_output = tree_to_dict(rst_tree_object)

        return jsonify(json_output)

    except Exception as e:
        return jsonify({"error": str(e)})

# --- THE STARTUP ---
def run_app():
    app.run(port=5000)

threading.Thread(target=run_app).start()


ngrok.set_auth_token("PANGITA SA IMONG NGROK TOKEN I-SEARCH RA")

# Open the connection
public_url = ngrok.connect(5000).public_url
print(f"\n🚀 YOUR SYSTEM IS LIVE AT: {public_url}")

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.



🚀 YOUR SYSTEM IS LIVE AT: https://unmirrored-maryjo-spokeless.ngrok-free.dev


In [ ]:
# --- THE SMARTER TRANSLATOR FUNCTION
def tree_to_dict(node):
    # 1. Check for Children FIRST
    children = []

    # Check Left Child
    if hasattr(node, 'left') and node.left:
        children.append(tree_to_dict(node.left))

    # Check Right Child
    if hasattr(node, 'right') and node.right:
        children.append(tree_to_dict(node.right))

    # 2. If we found children, this is a BRANCH
    if children:
        return {
            "type": "branch",
            "relation": getattr(node, 'relation', 'span'),
            "children": children
        }

    # 3. If NO children, only then is it a LEAF
    return {
        "type": "leaf",
        "relation": getattr(node, 'relation', 'elementary'),
        "text": getattr(node, 'text', '')
    }